In [ ]:
using Logging
logger = ConsoleLogger(stdout)
# debuglogger = ConsoleLogger(stderr, Logging.Debug)
global_logger(logger)
using JLD

using Random
rng = MersenneTwister(1234)
import Dates

In [ ]:
using Plots
pyplot()

In [ ]:
using ForwardDiff
using ProgressMeter
using LinearAlgebra: dot
include("utils/misc.jl")
using .MiscUtils: myCircle, mySphere, myDisk

In [ ]:
ts = Dates.now()

## model parameters
input_dim = 2
bias = true
m = 50  # nb hidden neurons for each sign
N = 5   # nb samples
n = 10  # nb particles per sample
rob = 0.2 # robustness level
activation_str = "ReLUcub"

## logging
ts_fsfriendly = Dates.format(ts, "yyyy-mm-ddTHHMMSS") # filesystem-friendly string for ts
resultdir = mkpath("results/DR__dim$(input_dim)bias$(bias)__m$(m)__N$(N)__n$(n)__rob$(rob)__$(activation_str)__$(ts_fsfriendly)")

logfile = "$resultdir/log.txt"
touch(logfile)
open(logfile, "a") do f
    write(f, "input_dim=$input_dim\n")
    write(f, "bias: $bias\n")
    write(f, "m=$m\n")
    write(f, "N=$N\n")
    write(f, "n=$n\n")
    write(f, "rob=$rob\n")
    write(f, "activation: $activation_str\n")
end

In [ ]:
alpha = 8
scaling = 1 # unkown what the "correct" scaling typically is!

## algo parameters
T = Int(floor(4000 / (alpha^scaling)))
eta0_a = 1e-1 * alpha # initial stepsize
eta0_u = 1e-3 * alpha
eta0_w = 1e-1 * alpha
eta0_theta = 1e-2 * alpha
constrain_theta = true
extrasteps = 2 # extrasteps=1: CP-MDA, extrasteps=2: CP-MP

open(logfile, "a") do f
    write(f, "T=$T\n")
    write(f, "eta0_a=$(eta0_a)\n")
    write(f, "eta0_u=$(eta0_u)\n")
    write(f, "eta0_w=$(eta0_w)\n")
    write(f, "eta0_theta=$(eta0_theta)\n")
    write(f, "constrain_theta: $(constrain_theta)\n")
    write(f, "extrasteps=$(extrasteps)\n")
end

In [ ]:
## plotting parameters
TNI_min = 1 # plot NI error
TNI_max = T
evalNIevery = Int(floor((TNI_max-TNI_min)/50))
Ntheta = Int(1e4)
Nu = Int(1e3)
skip_avg = true # avg not implemented

@assert input_dim==2
# r = range(-3, 3, length=101)
r = range(-2.2, 2.2, length=101)

Tplotreg_min = 1 # plot decision regions gif
Tplotreg_max = T
plotregevery = Int(floor((Tplotreg_max-Tplotreg_min)/50))
hidetitle = false
hidelabel = false
skip_gifs = true

In [ ]:
if activation_str == "ReLU"
    activation(x) = max(0, x)
elseif activation_str == "abs"
    activation(x) = abs(x)
elseif activation_str == "ReLUsq"
    activation(x) = max(0, x)^2
elseif activation_str == "ReLUcub"
    activation(x) = max(0, x)^3
elseif activation_str == "ReLUquar"
    activation(x) = max(0, x)^4
# elseif activation_str == "sq" # polynomial activation is silly
#     activation(x) = x^2
elseif activation_str == "sigmoid"
    activation(x) = 1/(1+exp(-x))
elseif activation_str == "tanh"
    activation(x) = tanh(x)
else
    error("activation_str not recognized, should be 
        \"ReLU\" or 
        \"abs\" or 
        \"ReLUsq\" or 
        \"ReLUcub\" or 
        \"ReLUquar\" or 
        \"sigmoid\" 
        or \"tanh\"")
end

if bias == true
    d = input_dim + 1
else
    d = input_dim
end

In [ ]:
function makedataset(d, N; bias, rng=MersenneTwister(1234))
    x = randn(rng, d, N)
    if bias
        x[d,:] .= 1
    end
    y = sign.(rand(rng, N).-0.5) # -1 or 1 uniformly
    return x, y
end

Random.seed!(rng, 1234)
x, y = makedataset(d, N; bias=bias, rng=rng)
save("$resultdir/dataset.jld", "x", x, "y", y)

plt = scatter(x[1, y.==1], x[2, y.==1], m=:circ, markersize=8, label="+", color=:green)
scatter!(x[1, y.==-1], x[2, y.==-1], m=:utriangle, markersize=8, label="-", color=:red)
for k=1:N
    circle = x[1:2, k] .+ rob .* myCircle(500)
    plot!(circle[1,:], circle[2,:], aspect_ratio=1.0, label=false, color=(y[k]==1 ? :green : :red))
end
display(plt)

In [ ]:
## min-max objective: obj(a, u, b, theta) = sum_{k=1}^N sum_{i=1}^n sum_{j=1}^m a_{ki} b_j neuronsigns[j] y_k sigma(theta_j.T (x_k+u_{ki}))
if bias
    # gfun(u, theta, k) = y[k] * activation(dot(theta[1:input_dim], x[1:input_dim,k] .+ u) + theta[d]*x[d,k]) # for some reason ForwardDiff doesn't like this
    gfun(u, theta, k) = y[k] * activation(dot(theta[1:2], x[1:2,k] .+ u) + theta[d]*x[d,k])
else
    gfun(u, theta, k) = y[k] * activation(dot(theta, x[:,k] .+ u))
end

function logit(x, w, theta)
    out = 0
    for j=1:m
        out += w[j] * activation(dot(theta[:,j], x))
    end
    for j=m+1:2m
        out += -w[j] * activation(dot(theta[:,j], x))
    end
    return out
end

if bias
    predict(x1, x2; w, theta, hard=true) = hard ? sign(logit([x1, x2, 1], w, theta)) : logit([x1, x2, 1], w, theta)
else
    predict(x1, x2; w, theta, hard=true) = hard ? sign(logit([x1, x2], w, theta)) : logit([x1, x2], w, theta)
end

In [ ]:
Random.seed!(rng, 1234)
# initialize adversary weights
a = ones(N, n) ./ (n*N)
u = randn(rng, input_dim, N, n) # (and not d, in case bias=true)
for k=1:N, i=1:n
    u[:,k,i] ./= sqrt(sum(u[:,k,i].^2))
    u[:,k,i] .*= rob * rand(rng)
end
# initialize network weights and positions nu=(w, theta); theta constrained to unit l2 sphere
w = ones(2m) ./ (2m)
theta = randn(rng, d, 2m)
for j=1:2m
    theta[:,j] ./= sqrt(sum(theta[:,j].^2))
end

# for extragradient step
ap = similar(a)
up = similar(u)
wp = similar(w)
thetap = similar(theta)
# to store intermediate values
copies_a = Array{Float64}(undef, N, n, T+1)
copies_u = Array{Float64}(undef, input_dim, N, n, T+1)
copies_w = Array{Float64}(undef, 2m, T+1)
copies_theta = Array{Float64}(undef, d, 2m, T+1)
;

In [ ]:
## plot the decision region at random initialization https://discourse.julialang.org/t/plotting-decision-boundary-regions-for-classifier/21397
plt = contour(r, r, 
    (x1, x2) -> predict(x1, x2; w=w, theta=theta, hard=false),
    f=true)
scatter!(x[1, y.==1], x[2, y.==1], m=:circ, markersize=8, label="+", color=:green)
scatter!(x[1, y.==-1], x[2, y.==-1], m=:utriangle, markersize=8, label="-", color=:red)
for k=1:N
    circle = x[1:2, k] .+ rob .* myCircle(500)
    plot!(circle[1,:], circle[2,:], #aspect_ratio=1.0, 
        label=false, color=(y[k]==1 ? :green : :red))
    scatter!(x[1,k].+u[1,k,:], x[2,k].+u[2,k,:], alpha=a[k,:], markersize=3, markerstrokewidth=0, label=false, color=(y[k]==1 ? :green : :red))
end
display(plt)

In [ ]:
"""
Take a CP gradient step
- starting from a, u, w, theta
- evaluating the gradients at ap, up, wp, thetap
- with stepsizes eta_a, eta_u, eta_w, eta_theta
(taking care of the fact that the network is parametrized such that the first m neurons are positively weighted and the last m are negatively weighted)
Returns the updated particles a1, u1, w1, theta1
"""
function step_CPMDA(
        f, rob,
        a, u, w, theta,
        ap, up, wp, thetap,
        eta_a, eta_u, eta_w, eta_theta;
        true_prox=false,
        constrain_theta=true
)
    N, n = size(a)
    d = size(theta)[1]
    m = Int(size(theta)[2] / 2)
    
    neuronsigns = ones(2m)
    neuronsigns[m+1:2m] .= -1

    Dtheta_fp = Array{Float64}(undef, d, N, n, 2m) # gradient of f w.r.t theta at thetap
    for k=1:N, i=1:n, j=1:2m
        Dtheta_fp[:,k,i,j] = ForwardDiff.gradient(tt -> f(up[:,k,i], tt, k), thetap[:,j])
    end
    Du_fp = Array{Float64}(undef, input_dim, N, n, 2m) # gradient of f w.r.t u at up
    for k=1:N, i=1:n, j=1:2m
        Du_fp[:,k,i,j] = ForwardDiff.gradient(uu -> f(uu, thetap[:,j], k), up[:,k,i])
    end

    # take step: adversary
    a1 = similar(a)
    u1 = similar(u)
    for k=1:N, i=1:n
        s = sum( neuronsigns[j] * wp[j] * f(up[:,k,i], thetap[:,j], k) for j=1:2m )
        a1[k,i] = a[k,i] * exp(-eta_a * s)
    end
    a1 ./= sum(a1)
    for k=1:N, i=1:n
        s = sum( neuronsigns[j] * wp[j] * Du_fp[:,k,i,j] for j=1:2m)
        @assert size(s) == (input_dim,)
        if true_prox
            u1[:,k,i] = u[:,k,i] - eta_u * ap[k,i] / a[k,i] * s
        else
            u1[:,k,i] = u[:,k,i] - eta_u * s
        end
    end
    # project u1 back to l2 ball of radius rob
    for k=1:N, i=1:n
        uki_sqnorm = sum(u1[:,k,i].^2)
        if uki_sqnorm > rob^2
            u1[:,k,i] ./= sqrt(uki_sqnorm)
            u1[:,k,i] .*= rob
        end
    end
    
    # take step: network
    w1 = similar(w)
    theta1 = similar(theta)
    for j=1:2m
        s = sum( ap[k,i] * f(up[:,k,i], thetap[:,j], k) for k=1:N, i=1:n )
        w1[j] = w[j] * exp(eta_w*neuronsigns[j]*s)
    end
    w1 ./= sum(w1)
    for j=1:2m
        s = sum( ap[k,i] * Dtheta_fp[:,k,i,j] for k=1:N, i=1:n )
        @assert size(s) == (d,)
        if true_prox
            theta1[:,j] = theta[:,j] + eta_theta * neuronsigns[j] * wp[j] / w[j] * s
        else
            theta1[:,j] = theta[:,j] + eta_theta * neuronsigns[j] * s
        end
    end
    # retract (just project) theta1 back to unit l2 sphere
    if constrain_theta
        for j=1:2m
            theta1[:,j] ./= sqrt(sum(theta1[:,j].^2))
        end
    end
    
    return a1, u1, w1, theta1
end


In [ ]:
eta_a, eta_u, eta_w, eta_theta = eta0_a, eta0_u, eta0_w, eta0_theta # constant stepsizes

@showprogress 1 for t=1:T # minimum update interval of 1 second
    copies_a[:,:,t] = copy(a)
    copies_u[:,:,:,t] = copy(u)
    copies_w[:,t] = copy(w)
    copies_theta[:,:,t] = copy(theta)

    # extragradient ("ghost" step)
    # extrasteps=1: CP-MDA, extrasteps=2: CP-MP
    ap, up, wp, thetap = a, u, w, theta
    for s=1:extrasteps
        ap, up, wp, thetap = step_CPMDA(
            gfun, rob,
            a, u, w, theta,
            ap, up, wp, thetap,
            eta_a, eta_u, eta_w, eta_theta;
            true_prox=true,
            constrain_theta=constrain_theta
        )
    end

    # take step
    a, u, w, theta = ap, up, wp, thetap
end

copies_a[:,:,T+1] = copy(a)
copies_u[:,:,:,T+1] = copy(u)
copies_w[:,T+1] = copy(w)
copies_theta[:,:,T+1] = copy(theta)

af, uf, wf, thetaf = a, u, w, theta
save("$resultdir/iterates.jld", "copies_a", copies_a, "copies_w", copies_w, "copies_theta", copies_theta)

In [ ]:
sum(af, dims=2)

In [ ]:
## plot the NI error
"""
Compute the "global" Nikaido-Isoda (NI) error
    max_{a0, nu0} <a|F|nu0> - <a0|F|nu> = max_theta <a|F|delta_theta> - min_i <ei|F|nu>
"""
function glob_NI_err(gfun, rob, a, u, w, theta; Ntheta=Int(1e4), Nu=Int(1e2), apcont_theta=nothing, apcont_nu=nothing)
    N, n = size(a)
    input_dim = size(u)[1]
    d = size(theta)[1]
    m = Int(size(theta)[2] / 2)
    
    neuronsigns = ones(2m)
    neuronsigns[m+1:2m] .= -1
    
    if !isnothing(apcont_theta)
        Ntheta_new = size(apcont_theta)[2]
    else
        if d==2
            apcont_theta = myCircle(Ntheta)
        elseif d==3
            apcont_theta, Ntheta_new = mySphere(Ntheta)
        else
            error("glob_NI_err not implemented for d>3")
        end
    end
    maxtheta = -Inf
    mintheta = +Inf
    for l=1:Ntheta_new
        theta0 = apcont_theta[:,l]
        s = sum( a[k,i] * gfun(u[:,k,i], theta0, k) for k=1:N, i=1:n )
        maxtheta = max(maxtheta, s)
        mintheta = min(mintheta, s)
    end

    if !isnothing(apcont_nu)
        Nu_new = size(apcont_nu)[2]
    else
        @assert input_dim==2
        apcont_nu, Nu_new = myDisk(Nu)
        apcont_nu .*= rob
    end
    minu = Inf
    for k=1:N # putting all mass on x_k + u for some u in B_{0,rob}
        for l=1:Nu_new
            u0 = apcont_nu[:,l]
            s = sum( neuronsigns[j] * w[j] * gfun(u0, theta[:,j], k) for j=1:2m )
            minu = min(minu, s)
        end
    end

    return max(maxtheta, -mintheta) - minu
end

In [ ]:
# TNI_min = 1
# TNI_max = 5000
# evalNIevery = Int(floor((TNI_max-TNI_min)/50))
# Ntheta = Int(1e5)
# Nu = Int(1e4)

In [ ]:
nierrs = Array{Float64}(undef, T+2)

if d==2
    apcont_theta = myCircle(Ntheta)
elseif d==3
    apcont_theta, Ntheta_new = mySphere(Ntheta)
else
    error("glob_NI_err not implemented for d>3")
end
@assert input_dim==2
apcont_nu, Nu_new = myDisk(Nu)
apcont_nu .*= rob

@showprogress 1 for t=TNI_min:evalNIevery:TNI_max+1 # minimum update interval of 1 second
# for t=TNI_min:evalNIevery:TNI_max+1
    nierrs[t] = glob_NI_err(gfun, rob, copies_a[:,:,t], copies_u[:,:,:,t], copies_w[:,t], copies_theta[:,:,t]; apcont_theta=apcont_theta, apcont_nu=apcont_nu) # Ntheta=Ntheta, Nu=Nu)
end
if !skip_avg
    nierrs[T+2] = glob_NI_err(gfun, rob, avg_a, avg_u, avg_w, avg_theta; apcont_theta=apcont_theta, apcont_nu=apcont_nu) # Ntheta=Ntheta, Nu=Nu)
end
open(logfile, "a") do f
    for t=TNI_min:evalNIevery:TNI_max+1
        write(f, "glob_NI_err at iteration#$t: $(nierrs[t])\n")
    end
    if !skip_avg 
        write(f, "glob_NI_err at avg iterate: $(nierrs[T+2])\n") 
    end
end
save("$resultdir/nierrs__every$(evalNIevery)__t=$(TNI_min)--$(TNI_max).jld", "nierrs", nierrs)

plt_NI = plot(range(TNI_min, stop=TNI_max+1, step=evalNIevery), nierrs[TNI_min:evalNIevery:TNI_max+1], xlabel="k", label="")
if !skip_avg
    hline!([nierrs[T+2]], label="avg iterate")
end
if !hidetitle
    title!("NI error of iterates")
end
fn = "$resultdir/NI_errors.png"
savefig(plt_NI, fn)

eps = 1e-10 # numerical stability (we use approximations (with deltax, deltay) to compute glob_NI_err)
plt_NI_log = plot(range(TNI_min, stop=TNI_max+1, step=evalNIevery), eps .+ max.(0, nierrs[TNI_min:evalNIevery:(TNI_max+1)]), xlabel="k", label="", yscale=:log10)
if !skip_avg
    hline!([ eps + max(0, nierrs[T+2]) ], label=(hidelabels ? "" : "avg iterate"))
end
if !hidetitle
    title!("NI error of iterates (log-linear scale)")
end
fn = "$resultdir/NI_errors_logscale.png"
savefig(plt_NI_log, fn)

plt_NI_log

In [ ]:
## decision region at the last iterate
pltt = contour(r, r,
    (x1, x2) -> predict(x1, x2; w=w, theta=theta, hard=false),
    f=true)
contour!(r, r,
    (x1, x2) -> predict(x1, x2; w=w, theta=theta, hard=false),
    levels=[0.],
    seriescolor=:blues,
    linestyle=:dash,
    linewidth=3)
scatter!(x[1, y.==1], x[2, y.==1], m=:circ, markersize=8, label="+", color=:green)
scatter!(x[1, y.==-1], x[2, y.==-1], m=:utriangle, markersize=8, label="-", color=:red)
for k=1:N
    circle = x[1:2, k] .+ rob .* myCircle(500)
    plot!(circle[1,:], circle[2,:], #aspect_ratio=1.0, 
        label=false, color=(y[k]==1 ? :green : :red))
    scatter!(x[1,k].+u[1,k,:], x[2,k].+u[2,k,:], 
        # alpha=a[k,:], 
        alpha=1,
        markersize=4, markerstrokewidth=0, label=false, color=(y[k]==1 ? :green : :red))
end

fn = "$resultdir/contour_soft_lastiter_withsatellites.png"
savefig(pltt, fn)
pltt

In [ ]:
## plot the neurons (are they sparsely concentrated?)
function plot_neurons_2d(w, theta; resultdir=nothing)
    d = size(theta)[1]
    m = Int(length(w)/2)
    @assert d==2
    r0 = 1/m
    circle = r0 .* myCircle(500)
    plt = plot(circle[1,:], circle[2,:], aspect_ratio=1.0, label=false)
    neur = zeros(2, 2m)
    for j=1:2m
        neur[:,j] = w[j] * theta[:,j]
    end
    for j=1:m
        plot!([0, neur[1,j]], [0, neur[2,j]], color=:red, label=false)
    end
    for j=m+1:2m
        plot!([0, neur[1,j]], [0, neur[2,j]], color=:blue, label=false)
    end
    scatter!(neur[1,1:m],    neur[2,1:m],    markersize=4, markercolor=:red, label=false)
    scatter!(neur[1,m+1:2m], neur[2,m+1:2m], markersize=4, markercolor=:blue, label=false)
    if !isnothing(resultdir)
        fn = "$resultdir/neurons_lastiter.png"
        savefig(plt, fn)
    end
    return plt
end

function plot_neurons_fake3d(w, theta; hidetitle=false, resultdir=nothing)
    plt_e1 = plot_neurons_2d(w, theta[ [2,3], :])
    xlabel!("theta_2")
    ylabel!("theta_3")
    if !hidetitle
        title!("neurons projected along axis theta_1")
    end
    plt_e2 = plot_neurons_2d(w, theta[ [1,3], :])
    xlabel!("theta_1")
    ylabel!("theta_3")
    if !hidetitle
        title!("neurons projected along axis theta_2")
    end
    plt_e3 = plot_neurons_2d(w, theta[ [1,2], :])
    xlabel!("theta_1")
    ylabel!("theta_2")
    if !hidetitle
        title!("neurons projected along axis theta_3")
    end
    plt_combined = plot(plt_e1, plt_e2, plt_e3, layout=(1, 3), size=(1500, 500))
    if !isnothing(resultdir)
        fn = "$resultdir/neurons_lastiter_projtheta1.png"
        savefig(plt_e1, fn)
        fn = "$resultdir/neurons_lastiter_projtheta2.png"
        savefig(plt_e2, fn)
        fn = "$resultdir/neurons_lastiter_projtheta3.png"
        savefig(plt_e3, fn)
        fn = "$resultdir/neurons_lastiter.png"
        savefig(plt_combined, fn)
    end
    plt_e1, plt_e2, plt_e3, plt_combined
end

In [ ]:
## neurons at last iterate
if d==3
    plt_e1, plt_e2, plt_e3, plt_combined = plot_neurons_fake3d(copies_w[:,T+1], copies_theta[:,:,T+1];
            hidetitle=hidetitle, resultdir=resultdir)
elseif d==2
    plt_combined = plot_neurons_2d(copies_w[:,T+1], copies_theta[:,:,T+1];
            resultdir=resultdir)
end
plt_combined

In [ ]:
copies_w[:,T+1]

In [ ]:
## plot the decision region across iterations (gif)
if !skip_gifs
    p = Progress(length(Tplotreg_min:plotregevery:Tplotreg_max))

    anim = @animate for t=Tplotreg_min:plotregevery:Tplotreg_max
        contour(r, r,
            (x1, x2) -> predict(x1, x2; w=copies_w[:,t], theta=copies_theta[:,:,t], hard=false),
            f=true)
        contour!(r, r,
            (x1, x2) -> predict(x1, x2; w=copies_w[:,t], theta=copies_theta[:,:,t], hard=false),
            levels=[0.],
            seriescolor=:blues,
            linestyle=:dash,
            linewidth=3)
        scatter!(x[1, y.==1], x[2, y.==1], m=:circ, markersize=8, label="+", color=:green)
        scatter!(x[1, y.==-1], x[2, y.==-1], m=:utriangle, markersize=8, label="-", color=:red)
        for k=1:N
            circle = x[1:2, k] .+ rob .* myCircle(500)
            plot!(circle[1,:], circle[2,:], #aspect_ratio=1.0, 
                label=false, color=(y[k]==1 ? :green : :red))
            scatter!(x[1,k].+copies_u[1,k,:,t], x[2,k].+copies_u[2,k,:,t], 
                # alpha=a[k,:], 
                alpha=1,
                markersize=4, markerstrokewidth=0, label=false, color=(y[k]==1 ? :green : :red))
        end
        title!("Iteration $t")
        next!(p)
    end

    fn = "$resultdir/contour_soft__every$(plotregevery)__$Tplotreg_min-$Tplotreg_max.gif"
    gif(anim, fn, fps=10)

    cp(anim.dir, resultdir*"/frames_contour_soft__every$(plotregevery)__$Tplotreg_min-$Tplotreg_max", force=true)
end